# Using the New Tool

## Code

In [ ]:
%%writefile viewscape.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: raster
# % keyword: statistics
# % keyword: viewshed
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# % guisection: Input
# %end
# %option G_OPT_M_COORDS
# % guisection: Input
# %end
# %option G_OPT_R_INPUTS
# % guisection: Input
# %end
# %option G_OPT_F_OUTPUT
# % guisection: Output
# %end
# %option
# % key: format
# % type: string
# % required: yes
# % options: json,csv
# % label: Output format
# % descriptions: json;JSON (JavaScript Object Notation);csv;CSV (Comma Separated Values)
# % answer: json
# % guisection: Output
# %end


import atexit
import subprocess
import sys
import csv
import json
import io

import grass.script as gs


def output_results(results, sample_rasters, file_name, file_format):
    if file_format == "json":
        with open(file_name, "w", encoding="utf-8") as json_file:
            json.dump(results, json_file, ensure_ascii=False, indent=4)
    elif file_format == "csv":
        with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
            header = ["name"]
            header.extend(results[sample_rasters[0]].keys())
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            for key, value in results.items():
                row = {"name": key}
                row.update(value)
                writer.writerow(row)
    else:
        raise ValueError(f"Unsupported or invalid format: {file_format}")


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def viewshed(
    elevation,
    coordinates,
    sample_rasters,
    output,
    file_format,
):
    viewshed = gs.append_node_pid("tmp_viewshed")
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
            "r.univar",
            map=name,
            zones=viewshed,
            quiet=True,
            flags="t",
            separator="comma",
        )
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    output_results(
        results=results,
        sample_rasters=sample_rasters,
        file_name=output,
        file_format=file_format,
    )


def main():
    options, flags = gs.parser()
    coordinates = options["coordinates"].split(",")
    sample_rasters = options["input"].split(",")
    viewshed(
        elevation=options["elevation"],
        coordinates=(float(coordinates[0]), float(coordinates[1])),
        sample_rasters=sample_rasters,
        output=options["output"],
        file_format=options["format"],
    )


if __name__ == "__main__":
    main()

As before, we will make the script executable:

In [ ]:
!chmod u+x viewscape.py

## Documentation

The command line help now looks like this (`--help` or `--h`):

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py --help

Running the script with `--html-description` gives the command line interface described in HTML which later becomes a part of the tool's HTML documentation:

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py --html-description > test.html

In [ ]:
from IPython.display import IFrame

IFrame("test.html", width=700, height=600)

## Desktop GUI

On desktop, a graphical user interface for the tool would be available, too, accessible, e.g., through `--ui`:

```bash
grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py --ui
```

The GUI window may look like this:

![GUI with tabs](img/gui_example_sections.png)

## Command Line Interface

The command line parameters in GRASS GIS are key-value pairs which are using syntax `key=value`. In the CLI world, this is sometimes called _named arguments_ and it is similar to Python keyword arguments.

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation coordinates=641583,226296 input=elevation,ndvi output="data.txt"

Try running the above again. The raster named _stations_ now exists, so GRASS GIS will automatically detect that and ask you to use `--overwrite` if you want to replace the existing data.

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation coordinates=641583,226296 input=elevation,ndvi output="data.txt"

With added `--overwrite` (or `--o`):

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation coordinates=641583,226296 input=elevation,ndvi output="data.txt" --overwrite

Let's view data range of the newly created raster:

In [ ]:
!cat data.txt

## Other interfaces

### WPS

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py --wps-process-description

### JSON

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation coordinates=641583,226296 input=elevation,ndvi output="data.txt" --json

## General Parameter Definition

To add general parameters such as text and numbers, we can use the following key-value syntax enclosed in `%option` and `%end`:

```python
# %option
# % key1: value1
# % key2: value2
# % key3: value3
# %end
```

Let's say we want to allow users of our tool to specify the raster value which is used where vector features are present. We will name it _value_ (`key: value`) and make it required (`required: yes`). The data type we will use is _double_ (`type: double`) which we can use as _float_ in Python. The following puts all these together:

```python
# %option
# % key: value
# % type: double
# % required: yes
# % description: Raster cell value where features are
# %end
```

## Using the New Tool from Python

The tool can be used from Python just like the other GRASS tools.

Here is a Python script which creates a GRASS session and calls our new tool:

In [ ]:
%%python
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.script.setup


def main():
    with grass.script.setup.init("~/grassdata/nc_spm_08_grass7/foss4g") as session:
        gs.run_command(
            "./viewscape.py",
            elevation="elevation",
            coordinates=(641_583,226_296),
            input=["elevation", "ndvi"],
            output="data.json",
            overwrite=True,  # So that we can execute the notebook again.
        )


if __name__ == "__main__":
    main()

## Using Existing Interfaces for Generating Wrappers and Boilerplates

Often, a new tool is somehow wrapping or extending an existing tool or is similar to one. To quickly generate a boilerplate code in such cases, we can run any GRASS tool with `--script`. Unfortunately, `--script` does not currently output standard options, so the generated definitions are unnecessarily complicated.

Given that the same structure is needed every time, it is a good idea to use `--script` or copy-paste code from existing tools or examples. In the GRASS GIS source code, the Python scripts are under _[scripts](https://github.com/OSGeo/grass/tree/releasebranch_8_2/scripts)_ and _[temporal](https://github.com/OSGeo/grass/tree/releasebranch_8_2/temporal)_. Tools in the grass-addons repository are not organized by language, but many of the tools are in Python.

Here is how to get a Python script boilerplate from _v.to.rast_ (which itself is in written C):

In [ ]:
!grass --tmp-location XY --exec v.to.rast --script